In [1]:
library(hipathia)
library(circlize)
library(ComplexHeatmap)
library(ggplot2)
library(corrplot)
library(hipathia)
library(reshape2)
library(dplyr)
library(ggdendro)
library(limma)

Loading required package: igraph


Attaching package: ‘igraph’


The following objects are masked from ‘package:stats’:

    decompose, spectrum


The following object is masked from ‘package:base’:

    union


Loading required package: AnnotationHub

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:igraph’:

    normalize, path, union


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, aperm, append, as.data.frame, basename, cbind,
    colnames, dirname, do.call, duplicated, eval, evalq, Filter, Find,
    get, grep, grepl, intersect, is.unsorted, lapply, Map, mapply,
    match, mget, order, paste, pmax, pmax.int, pmin, pmin.int,
    Position, rank, rbind, Reduce, rownames, sapply, setdiff, sort,
    table, tapply, union, unique, unsplit, which.max, which.min


Loading required package: BiocFileCache



In [2]:
annot_splattrib <- read.table("/mnt/lustre/scratch/CBRA/research/projects/heterogeneity_mm/data/GTEx_Analysis_v8_Annotations_SampleAttributesDS.txt", sep="\t", header=T)
pathvals_gtex <- readRDS("/mnt/lustre/scratch/CBRA/research/projects/heterogeneity_mm/results/00_mm_gtex/pathvals_gtexv8.rds")
annot_splattrib_comp <- annot_splattrib[annot_splattrib$SAMPID %in% colnames(pathvals_gtex),]
annot_splattrib_comp <- annot_splattrib_comp[annot_splattrib_comp$SMTSD != "",]

pathways <- load_pathways("hsa")
pathvals <- assay(pathvals_gtex)
path_names <- get_path_names(pathways,rownames(pathvals))
rownames(pathvals) <- path_names

annot_subph <- read.table("/mnt/lustre/scratch/CBRA/research/projects/heterogeneity_mm/data/GTEx_Analysis_v8_Annotations_SubjectPhenotypesDS.txt", sep="\t", header=T)
rownames(annot_subph) <- annot_subph$SUBJID

Warning message in scan(file = file, what = what, sep = sep, quote = quote, dec = dec, :
“EOF within quoted string”
Warning message in scan(file = file, what = what, sep = sep, quote = quote, dec = dec, :
“number of items read is not a multiple of the number of columns”
Loaded 146 pathways



In [3]:
annot_splattrib_sl <- annot_splattrib[,c("SAMPID","SMTS","SMTSD")]
rownames(annot_splattrib_sl) <- annot_splattrib_sl$SAMPID

comp_pat <- intersect(annot_splattrib_sl$SAMPID, colnames(pathvals))
subjib_comp_pat <- paste("GTEX-",unique(unlist(lapply(strsplit(comp_pat,"-", fixed=T), FUN=function(x) x[2]))), sep="")

annot_subph_comp <- annot_subph[subjib_comp_pat,]                                                                 
list_patient <- lapply(annot_subph_comp$SUBJID, FUN=function(x) annot_splattrib_sl$SAMPID[grep(x, annot_splattrib_sl$SAMPID)])
list_sex <- do.call(list,lapply(1:length(annot_subph_comp$SEX), function(x) rep(annot_subph_comp$SEX[x],length(unlist(list_patient[x])))))                       
list_age <- do.call(list,lapply(1:length(annot_subph_comp$AGE), function(x) rep(annot_subph_comp$AGE[x],length(unlist(list_patient[x])))))   
pat_sex_age <- data.frame(patients = unlist(list_patient),
                                   sex = unlist(list_sex),
                                   age = unlist(list_age))
rownames(pat_sex_age) <- pat_sex_age$patients

pat_sex_age_comp <- pat_sex_age[comp_pat,]
pat_sex_age_comp$sex[which(pat_sex_age_comp$sex == 2)] <- "female"
pat_sex_age_comp$sex[which(pat_sex_age_comp$sex == 1)] <- "male"
pat_sex_age_comp$tissue <- annot_splattrib_sl[pat_sex_age_comp$patients,"SMTS"]                                
pat_sex_age_comp$tissue_zone <- annot_splattrib_sl[pat_sex_age_comp$patients,"SMTSD"] 
annot_comp <- pat_sex_age_comp[,c("patients","age","sex","tissue","tissue_zone")]    
annot_comp <- annot_comp[annot_comp$tissue_zone != "",]

In [4]:
annot_comp_fcells <- annot_comp[-c(which(annot_comp$tissue_zone == 'Cells - Cultured fibroblasts'), which(annot_comp$tissue_zone == 'Cells - EBV-transformed lymphocytes')),]
pathvals_fcells <- pathvals[,annot_comp[-c(which(annot_comp$tissue_zone == 'Cells - Cultured fibroblasts'), which(annot_comp$tissue_zone == 'Cells - EBV-transformed lymphocytes')),"patients"]]

In [5]:
pca.obj <- prcomp(t(pathvals_fcells))

tissue <- annot_comp_fcells$tissue
dtp <- data.frame('Tipos' = tissue, pca.obj$x[,1:2]) # the first two componets are selected (NB: you can also select 3 for 3D plottings or 3+)

In [6]:
jpeg("/mnt/lustre/scratch/CBRA/research/projects/heterogeneity_mm/results/01_path_more_carac_tissue/pca_tissues.jpeg", quality = 100, width = 600, height = 400)
ggplot(data = dtp) + 
  geom_point(aes(x = PC1, y = PC2, col = tissue)) + scale_color_manual(values = c(  
 "Adipose Tissue" = "#FF0000",
 "Muscle" = "#00FF00",
 'Blood Vessel' = "#0000FF",
 "Heart" = "#9932CC",
 "Uterus" = "#00FFFF",
 "Vagina" = "#FF00FF",  
 "Breast" = "#FFFF00",
 "Skin" = "#FFA500",
 "Salivary Gland" = "#FFC0CB", 
 "Brain" = "#800080",
 "Adrenal Gland" = "#A52A2A",
 "Thyroid" = "#808000",
 "Lung" = "#008080",
 "Spleen" ="#800000",
 "Pancreas" = "#000080",
 "Esophagus" = "#E6E6FA",
 "Stomach" = "#40E0D0",
 "Colon" = "#FFD700",
 "Small Intestine" = "#C0C0C0",
 "Prostate" = "#4B0082",
 "Testis" = "#708090",
 "Nerve" = "#FA8072",
 "Liver" = "#228B22",
 "Ovary" = "#87CEEB",
 "Pituitary" = "#191970",
 "Whole Blood" = "#9932CC",
 "Kidney" = "#FF6347")) + guides(color = guide_legend(nrow = 7)) + ggtitle("")+ theme_minimal() + theme(legend.position="bottom", legend.text = element_text(size=12))
dev.off()

png 
  2

In [7]:
#### pathways more characteristic of each tissue

In [8]:
tissue <- unique(annot_comp_fcells$tissue)
pathvals_mean <- as.data.frame(lapply(tissue, function(x) apply(pathvals_fcells[,annot_comp_fcells[which(x== annot_comp_fcells$tissue), "patients"]], 1, mean)))
colnames(pathvals_mean) <- tissue
rownames(pathvals_mean) <- path_names                 

pathvals_mean$circ <- rownames(pathvals_mean)
melt_pathvals_mean <- melt(pathvals_mean, id.vars = "circ")
pathvals_mean$circ <- NULL

In [9]:
physiological_paths <- read.table("/mnt/lustre/scratch/CBRA/research/projects/heterogeneity_mm/data/physiological_paths.tsv", sep="\t") 

sum_mean_pathvals <- apply(pathvals_mean,1,sum)

sum_mean_paths <- lapply(physiological_paths$V1, FUN=function(x) sum(sum_mean_pathvals[grep(x,names(sum_mean_pathvals))]))
sum_pathvals_tissue <- do.call(rbind, lapply(physiological_paths$V1, FUN=function(x) unlist(apply(pathvals_mean[grep(x, rownames(pathvals_mean)),],2,sum))))
rownames(sum_pathvals_tissue) <- physiological_paths$V1

sum_mean_paths <- data.frame("path_fisio"=physiological_paths$V1, "sum_mean_pathvals"=unlist(sum_mean_paths))
rownames(sum_mean_paths) <- sum_mean_paths$path_fisio

porc_sum_pathvals_tissue <- sum_pathvals_tissue/sum_mean_paths$sum_mean_pathvals *100

max_porc_sum_pathvals_tissue <- apply(porc_sum_pathvals_tissue, 1, max)
tissue_max_porc_sum_pathvals <- data.frame("rutas"=physiological_paths$V1, "tissue_mayor_porc"=unlist(lapply(physiological_paths$V1, FUN=function(x) names(which(porc_sum_pathvals_tissue[x,] == max_porc_sum_pathvals_tissue[x])))))

tissue <- unique(tissue_max_porc_sum_pathvals$tissue_mayor_porc)

sum_tej_paths <- data.frame("tissue"=tissue, "rutas"=unlist(lapply(tissue, FUN=function(x) 
paste(tissue_max_porc_sum_pathvals[which(x == tissue_max_porc_sum_pathvals$tissue_mayor_porc),"rutas"], collapse=", "))))
sum_tej_paths

tissue,rutas
<chr>,<chr>
Liver,"PPAR signaling pathway, AMPK signaling pathway, Complement and coagulation cascades, Glucagon signaling pathway"
Blood,"MAPK signaling pathway, Chemokine signaling pathway, NF-kappa B signaling pathway, Sphingolipid signaling pathway, p53 signaling pathway, mTOR signaling pathway, Notch signaling pathway, Platelet activation, Antigen processing and presentation, Toll-like receptor signaling pathway, NOD-like receptor signaling pathway, RIG-I-like receptor signaling pathway, Cytosolic DNA-sensing pathway, Jak-STAT signaling pathway, Natural killer cell mediated cytotoxicity, T cell receptor signaling pathway, B cell receptor signaling pathway, Fc epsilon RI signaling pathway, Fc gamma R-mediated phagocytosis, TNF signaling pathway"
Skin,"ErbB signaling pathway, Apoptosis, Adherens junction, Melanogenesis"
Brain,"Ras signaling pathway, Axon guidance, Circadian entrainment, Retrograde endocannabinoid signaling, Glutamatergic synapse, Cholinergic synapse, Serotonergic synapse, Dopaminergic synapse, Insulin secretion"
Adipose Tissue,"Rap1 signaling pathway, Signaling pathways regulating pluripotency of stem cells, Regulation of lipolysis in adipocytes"
Heart,"Calcium signaling pathway, HIF-1 signaling pathway"
Blood Vessel,"cGMP-PKG signaling pathway, Vascular smooth muscle contraction, Hippo signaling pathway, Focal adhesion, Neurotrophin signaling pathway, Thyroid hormone signaling pathway, Oxytocin signaling pathway"
Testis,"cAMP signaling pathway, Cell cycle, GABAergic synapse"
Muscle,"FoxO signaling pathway, Longevity regulating pathway - mammal, TGF-beta signaling pathway, Circadian rhythm, Insulin signaling pathway, Adipocytokine signaling pathway, Carbohydrate digestion and absorption"


In [10]:
tissues <- unique(annot_comp_fcells$tissue)
physiological_paths <- read.table("/mnt/lustre/scratch/CBRA/research/projects/heterogeneity_mm/data/physiological_paths.tsv", sep="\t") 
pathvals_fcells_phys <- pathvals_fcells[unlist(lapply(physiological_paths$V1, FUN=function(x) grep(x, rownames(pathvals_fcells)))),]

In [11]:
funct_limma_tissues_circ_fq95_logfc <- function(tissue_1,tissue_2){

    if (tissue_1 != tissue_2){
        type <- c(rep(tissue_1, length(annot_comp_fcells[annot_comp_fcells$tissue == tissue_1,"patients"])), rep(tissue_2, length(annot_comp_fcells[annot_comp_fcells$tissue == tissue_2,"patients"])))
        paths <- pathvals_fcells_phys[,c(annot_comp_fcells[annot_comp_fcells$tissue == tissue_1,"patients"],annot_comp_fcells[annot_comp_fcells$tissue == tissue_2,"patients"])]

        design <- model.matrix(~type)
        fit <- lmFit(paths, design)
        fit2 <- eBayes(fit)
        top_table <- topTable(fit2, number = Inf, p.value = 1, adjust="fdr")

        top_table$tissues <- rep(paste(tissue_1, " - ", tissue_2, sep=""), length(rownames(paths)))
        top_table_sign <- top_table[top_table$adj.P.Val < 0.05,]
        top_table_sign_flogfc <- top_table_sign[abs(top_table_sign$logFC) > quantile(abs(top_table_sign$logFC), probs=seq(0,1, by=0.05))[20],]
        top_table_sign_flogfc$circ <- rownames(top_table_sign_flogfc)
        return(rownames(top_table_sign_flogfc))
    }
}   

top_table_limma_tissue_fq95_logfc <- lapply(tissues, FUN=function(y) lapply(tissues, FUN=function(x) funct_limma_tissues_circ_fq95_logfc(x, y)))    

Removing intercept from test coefficients

Removing intercept from test coefficients

Removing intercept from test coefficients

Removing intercept from test coefficients

Removing intercept from test coefficients

Removing intercept from test coefficients

Removing intercept from test coefficients

Removing intercept from test coefficients

Removing intercept from test coefficients

Removing intercept from test coefficients

Removing intercept from test coefficients

Removing intercept from test coefficients

Removing intercept from test coefficients

Removing intercept from test coefficients

Removing intercept from test coefficients

Removing intercept from test coefficients

Removing intercept from test coefficients

Removing intercept from test coefficients

Removing intercept from test coefficients

Removing intercept from test coefficients

Removing intercept from test coefficients

Removing intercept from test coefficients

Removing intercept from test coefficients

Removing in

In [12]:
tis_1 <- unlist(lapply(1:length(tissues), FUN=function(x) rep(tissues[x], length(unlist(top_table_limma_tissue_fq95_logfc[x])))))
tis_2 <- unlist(lapply(1:length(tissues), FUN=function(x)  lapply(1:length(tissues), FUN=function(y) rep(tissues[y], length(unlist(top_table_limma_tissue_fq95_logfc[[x]][y]))))))                       

circ_limma_tis_fc95_logfc <- data.frame(
"tissue_1"=tis_1,
"tissue_2"=tis_2,
"circs"= unlist(top_table_limma_tissue_fq95_logfc)
      )                                                                  

In [24]:
jpeg("/mnt/lustre/scratch/CBRA/research/projects/heterogeneity_mm/results/01_path_more_carac_tissue/heatmap_circs_tis_fc95_logfc.jpeg", quality = 100, width = 600, height = 800)

circ_limma_tis_fc95_logfc %>% group_by(circs, tissue_1) %>% summarise(n=n()) %>% 
filter(n > 25) %>% 
ggplot(aes(x= tissue_1, y = circs, fill = n)) +
  geom_tile() + scale_fill_gradient(low = "#CCFF99",
    high = "#008280",
    na.value = "grey80") +
  scale_x_discrete(name="Tissues",expand = expansion(mult = c(0,0)), guide = guide_axis(angle = 90)) +  scale_y_discrete(name="Physiological circuits") + ggtitle("") + 
labs(fill=paste("Sig","dif",sep="\n")) + theme_minimal() +theme(axis.text.y= element_text(size=12), axis.text.x= element_text(size=12))  

dev.off()

`summarise()` has grouped output by 'circs'. You can override using the
`.groups` argument.


png 
  2

In [49]:
length(unique(data.frame(circ_limma_tis_fc95_logfc %>% group_by(circs, tissue_1) %>% summarise(n=n()) %>% 
filter(n > 25))$circs))
length(tissues)

`summarise()` has grouped output by 'circs'. You can override using the `.groups` argument.


[1] 37

[1] 27